In [ ]:
import torch
print(torch.__version__)
print(torch.version.cuda)

In [ ]:
!pip install --no-index torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cpu.html
!pip install --no-index torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cpu.html
!pip install --no-index torch-cluster -f https://pytorch-geometric.com/whl/torch-1.7.0+cpu.html
!pip install --no-index torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.7.0+cpu.html
!pip install torch-geometric

In [ ]:
from torch_geometric.data import Data

# data.edge_index: Graph connectivity in COO format with shape [2, num_edges] and type torch.long
# if in shape [num_edges, 2]: using edge_index.t().contiguous() to transform the tensor
edge_index = torch.tensor([[0, 1, 1, 2],[1, 0, 2, 1]], dtype=torch.long)
# data.x: Node feature matrix with shape [num_nodes, num_node_features]
# all the nodes here only have 1 feature.
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)
# data.edge_attr: Edge feature matrix with shape [num_edges, num_edge_features]
# data.pos: Node position matrix with shape [num_nodes, num_dimensions]
# the features for nodes and edges are kept separately
# None of these attributes is required, for example, when edges are not provided with attributes, edge_attr is None
# create a graph with defined nodes and edges
data = Data(x=x, edge_index=edge_index)

# some attributes about the graph
print("Keys in the graph dict:", data.keys)
print("Feature num for each node:", data.num_node_features)
print("The num of nodes:", data.num_nodes)
print("The num of edges", data.num_edges)
# pre-built functions
print("Isolation:", data.contains_isolated_nodes())
print("Self loop:", data.contains_self_loops())
print("Is directed:", data.is_directed())

In [ ]:
from torch_geometric.datasets import TUDataset

# load the ENZYMES sample dataset
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')
print("Graph num:", len(dataset))
print("Class num:", dataset.num_classes)
print("Feature num for each node:", dataset.num_node_features)
# select one graph after shuffling
data = dataset.shuffle()[-1]
print("Is directed:", data.is_directed())
print("Graph info:", data)
# the shape of y(target) is [1], so ENZYMES dataset can be used for graph classification(embedding) traing: graph-level

In [ ]:
from torch_geometric.datasets import Planetoid

# load the Cora dataset: node-level
dataset = Planetoid(root='/tmp/Cora', name='Cora')
print("Graph num:", len(dataset))
print("Class num:", dataset.num_classes)
print("Feature num for each node:", dataset.num_node_features)
data = dataset[0]
print("Graph info:", data)
# it's a single, undirected citation graph
print("The num of masked items in training set:", data.train_mask.sum().item())

In [ ]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# build a small GCN for node classification
class NodeClsNet(torch.nn.Module):
    def __init__(self, dataset, hidden_dim):
        super(NodeClsNet, self).__init__()
        # two GCN conv layers
        self.conv1 = GCNConv(dataset.num_node_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # propagating
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        # output logits
        return F.log_softmax(x, dim=1)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# prepare the model
model = NodeClsNet(dataset, 32).to(device)
data = dataset.shuffle()[-1].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
model.train()
# try to train the model
for epoch in range(200):
    out = model(data)
    # the -LL loss
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    optimizer.zero_grad()
    # backpropagation
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()
_, pred = model(data).max(dim=1)
# correct the num of right classified samples
correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / int(data.test_mask.sum())
print('Accuracy: {:.4f}'.format(acc))

In [ ]:
# Generalizing the convolution operator to irregular domains is typically expressed as a neighborhood aggregation or message passing scheme.
# how to define the neighbourhoods and how to aggregate data from them are the most important qestions when designing GCNs
# the choosen of aggregation mehtods and neighbourhoods reflect the basis you are try to applying, like the Laplacian or wavlet basis
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

# implement an GCN conv layer
# neighboring node features are first transformed by a weight matrix Θ, normalized by their degree, and finally, aggregation
class CustomGCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        # i.e. aggr="add", aggr="mean" or aggr="max".
        super(CustomGCNConv, self).__init__(aggr='add') 
        # the layers to adjust the neighbours' embeddings before aggregation
        self.lin = torch.nn.Linear(in_channels, out_channels)
        
    def forward(self, x, edge_index):
        # Step 1: Add self-loops to the adjacency matrix (nodes can reach themselves)
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)
        # Step 3: Compute normalization.
        row, col = edge_index
        # the D matrix: degree of each node, this process can be put into the message function
        deg = degree(col, x.size(0), dtype=x.dtype)
        # The normalization coefficients are derived by the node degrees deg(i) for each node i which gets transformed to 1/(deg(i)^(1/2)⋅deg(j)^(1/2)) for each edge (j,i)∈E.
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        # Step 4-5: Start propagating messages, this will activate the message function automatically
        # x has the adjusted embeddings of neighbours and norm is calculated for normalisation
        return self.propagate(edge_index, x=x, norm=norm)

    def message(self, x_j, norm):
        # Step 4: Normalize node features.
        # normalise each node: point-wise fashion
        return norm.view(-1, 1) * x_j

In [ ]:
# Implementing the Edge Convolution
class CustomEdgeConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(CustomEdgeConv, self).__init__(aggr='max')
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(2 * in_channels, out_channels),
            torch.nn.ReLU(),
            torch.nn.Linear(out_channels, out_channels)
        )

    def forward(self, x, edge_index):
        # not adjust the attributes of edges
        return self.propagate(edge_index, x=x)

    def message(self, x_i, x_j):
        # concate extra info to the target edge
        tmp = torch.cat([x_i, x_j - x_i], dim=1)  # tmp has shape [E, 2 * in_channels]
        return self.mlp(tmp)
    
from torch_geometric.nn import knn_graph

# edge convolution is actually a dynamic convolution, which recomputes the graph for each layer using nearest neighbors in the feature space
# edge convolution should be useful when doing the graph-level job: summarize the info of subgraphs
class DynamicEdgeConv(CustomEdgeConv):
    def __init__(self, in_channels, out_channels, k=6):
        super(DynamicEdgeConv, self).__init__(in_channels, out_channels)
        self.k = k

    def forward(self, x, batch=None):
        # torch_geometric.nn.pool.knn_graph(): a GPU accelerated batch-wise k-NN graph generation method 
        edge_index = knn_graph(x, self.k, batch, loop=False, flow=self.flow)
        return super(DynamicEdgeConv, self).forward(x, edge_index)

In [ ]:
import os.path as osp
from torch_geometric.data import Dataset

# create a customized dataset
class MyOwnDataset(Dataset):
    # the pre_transform function can: split the graph into subgraphs with knn method or etc.
    # once defined the pre_transform function and save the data, it will automatically do the pre_transform when loaded again.
    def __init__(self, root, transform=None, pre_transform=None):
        super(MyOwnDataset, self).__init__(root, transform, pre_transform)

    @property
    def raw_file_names(self):
        # may split the training files into serval parts
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        # data after transform/process
        return ['data_1.pt', 'data_2.pt', ...]

    # You can skip downloading and/or processing by just not overriding the download() and process() methods
    def download(self):
        # Download to `self.raw_dir`.

    def process(self):
        i = 0
        for raw_path in self.raw_paths:
            # Read data from `raw_path`.
            data = Data(...)
            if self.pre_filter is not None and not self.pre_filter(data):
                continue
            if self.pre_transform is not None:
                data = self.pre_transform(data)
            torch.save(data, osp.join(self.processed_dir, 'data_{}.pt'.format(i)))
            i += 1

    def len(self):
        return len(self.processed_file_names)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, 'data_{}.pt'.format(idx)))
        return data